In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import time
import csv
from torch import nn as nn
from torch.nn.functional import normalize
from PIL import Image
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset
import torch.utils.data.dataloader as dataloader
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from skimage import io
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [2]:
data_path = pd.DataFrame(pd.read_csv('dataset_csv_file.csv'))
data_path.head()

,Btop,Bjungle,Bmid,Bbot,Bsupport,Rtop,Rjungle,Rmid,Rbot,Rsupport,Bwin
0,95,79,31,46,85,16,82,17,27,3,1
1,118,79,65,15,100,40,37,17,47,66,0
2,68,82,115,27,3,112,63,65,100,85,0
3,78,103,83,48,77,95,37,17,98,74,0
4,29,81,40,46,50,93,25,58,47,74,0


In [3]:
data = data_path[["Btop", 'Bjungle', 'Bmid', 'Bbot', 'Bsupport', 'Rtop', 'Rjungle',
       'Rmid', 'Rbot', 'Rsupport']]
target = data_path['Bwin']
#data.shape, target.shape, print(data), print(target)

In [4]:
data_path_n = MinMaxScaler().fit_transform(data_path)
X = data_path_n[:,0:10]
y = data_path_n[:,10]
#data.shape, target.shape, print(X), print(y)

In [56]:
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=42)

sc = StandardScaler()

x_train_sc = sc.fit_transform(x_train)
x_test_sc = sc.transform(x_test)

Y_train = np.array(y_train)
Y_test = np.array(y_test)

x_train_sc.shape, x_test_sc.shape, y_train.shape, y_test.shape

((3222, 10), (806, 10), (3222,), (806,))

In [57]:
from collections import Counter
Counter(y_train), Counter(y_test)

(Counter({0.0: 2481, 1.0: 741}), Counter({0.0: 622, 1.0: 184}))

In [58]:
X_train = torch.from_numpy(x_train_sc.astype(np.float32))
X_test = torch.from_numpy(x_train_sc.astype(np.float32))
Y_train = torch.from_numpy(Y_train.astype(np.float32))
Y_test = torch.from_numpy(Y_test.astype(np.float32))

In [59]:
Y_train = Y_train.view(Y_train.shape[0],1)
Y_test = Y_train.view(Y_train.shape[0],1)

In [60]:
n_samples, n_features = X.shape

class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

learning_rate = 1e-3

loss_fn = nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [61]:
n_epochs = 3000

t1 = time.process_time()
for epoch in range(1, n_epochs + 1):
    
   
    
    y_predicted = model(X_train)
    loss = loss_fn(y_predicted, Y_train)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    #if (epoch+1) % 10 == 0:
        #print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
    
    print(f"Epoch: %d, Loss: %f" % (epoch,float(loss)))
    
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(Y_test).sum() / float(Y_test.shape[0])
    print(f'accuracy = {acc:.4f}')
t2 = time.process_time()
print(f'Time: {t2-t1} seconds')

Epoch: 1, Loss: 0.710639
Epoch: 2, Loss: 0.710558
Epoch: 3, Loss: 0.710478
Epoch: 4, Loss: 0.710398
Epoch: 5, Loss: 0.710318
Epoch: 6, Loss: 0.710239
Epoch: 7, Loss: 0.710159
Epoch: 8, Loss: 0.710079
Epoch: 9, Loss: 0.709999
Epoch: 10, Loss: 0.709919
Epoch: 11, Loss: 0.709839
Epoch: 12, Loss: 0.709760
Epoch: 13, Loss: 0.709680
Epoch: 14, Loss: 0.709600
Epoch: 15, Loss: 0.709521
Epoch: 16, Loss: 0.709441
Epoch: 17, Loss: 0.709362
Epoch: 18, Loss: 0.709282
Epoch: 19, Loss: 0.709203
Epoch: 20, Loss: 0.709123
Epoch: 21, Loss: 0.709044
Epoch: 22, Loss: 0.708965
Epoch: 23, Loss: 0.708885
Epoch: 24, Loss: 0.708806
Epoch: 25, Loss: 0.708727
Epoch: 26, Loss: 0.708647
Epoch: 27, Loss: 0.708568
Epoch: 28, Loss: 0.708489
Epoch: 29, Loss: 0.708410
Epoch: 30, Loss: 0.708331
Epoch: 31, Loss: 0.708252
Epoch: 32, Loss: 0.708173
Epoch: 33, Loss: 0.708094
Epoch: 34, Loss: 0.708015
Epoch: 35, Loss: 0.707936
Epoch: 36, Loss: 0.707857
Epoch: 37, Loss: 0.707778
Epoch: 38, Loss: 0.707700
Epoch: 39, Loss: 0.70

In [62]:
from sklearn.linear_model import LogisticRegression

#C = [.1, 1e-2, 1e-3, 1e-4]

#for c in C:
    
model_log = LogisticRegression(penalty='l2',C=1e-3,solver='liblinear',class_weight='balanced')
model_log.fit(x_train_sc, y_train)
expected = y_test
predicted = model_log.predict(x_test_sc)
probability = model_log.predict_proba(x_test_sc)[:,1]

trainAcc = model_log.score(x_train_sc, y_train)
valAcc = model_log.score(x_test_sc, y_test)

#print('C: ', c)
print('Training Accuracy:',trainAcc)
print('Validation Accuracy:',valAcc)

Training Accuracy: 0.5189323401613904
Validation Accuracy: 0.5310173697270472


In [63]:
print(metrics.classification_report(expected, predicted, digits=4))
print(metrics.confusion_matrix(expected, predicted))

              precision    recall  f1-score   support

         0.0     0.7877    0.5370    0.6386       622
         1.0     0.2461    0.5109    0.3322       184

    accuracy                         0.5310       806
   macro avg     0.5169    0.5239    0.4854       806
weighted avg     0.6641    0.5310    0.5687       806

[[334 288]
 [ 90  94]]


In [64]:
model_nb = GaussianNB()
model_nb.fit(x_train_sc, y_train)
expected_nb = y_test
predicted_nb = model_nb.predict(x_test_sc)
probablility_nb = model_nb.predict_proba(x_test_sc)

In [65]:
print(metrics.classification_report(expected_nb, predicted_nb, digits=3))
print(metrics.confusion_matrix(expected_nb, predicted_nb))

              precision    recall  f1-score   support

         0.0      0.772     1.000     0.871       622
         1.0      0.000     0.000     0.000       184

    accuracy                          0.772       806
   macro avg      0.386     0.500     0.436       806
weighted avg      0.596     0.772     0.672       806

[[622   0]
 [184   0]]


C:\Users\Cato\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Cato\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Cato\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
